# CHAPTER 5. MDP를 모를 때 value 평가하기
TD(Temporal Difference)

MC의 단점으로 재귀적인 특징을 갖고 있어, 에피소드가 끝나야 return값을 얻어 업데이트를 할 수 있다.   
TD는 상태 가치의 기대값을 예측하고, 예측한 기대값을 토대로 다음 번 상태 가치의 기대값을 예측한다.   
이러한 방식은 값을 한 개 씩 얻을 때마다 업데이트 할 수 있다.

$MC : V(s_t) ← V(s_t) + \alpha(G_t-V(s_t))$   
$TD : V(s_t) ← V(S_t) + \alpha(r_{t+1}+\gamma V(s_{t+1})-V(s_t))$

---

$V(s_0) ← V(s_0)+0.01*(-1+V(s_1)-V(s_0))$   
$V(s_1) ← V(s_1)+0.01*(-1+V(s_2)-V(s_1))$

### pseudo-code
> 1. 테이블의 값을 초기화 한다.
2. agent가 policy에 따라 경험을 쌓는다.
3. 상태 전이가 일어나면 테이블의 값을 업데이트 해준다.
4. 에피소드를 마칠 때 까지 2~3번을 반복한다.
5. 테이블의 값이 수렴할 때 까지 4번을 반복한다.
6. 수렴한 테이블의 결과를 출력해준다.

### grid world; env

In [1]:
import random
import numpy as np


class GridWorld():
    def __init__(self):
        self.x = 0
        self.y = 0

    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션: 아래쪽
        if a == 0:
            self.move_left()
        elif a == 1:
            self.move_up()
        elif a == 2:
            self.move_right()
        elif a == 3:
            self.move_down()

        reward = -1  # 보상은 항상 -1로 고정
        done = self.is_done()
        return (self.x, self.y), reward, done

    def move_right(self):
        self.y += 1
        if self.y > 3:
            self.y = 3

    def move_left(self):
        self.y -= 1
        if self.y < 0:
            self.y = 0

    def move_up(self):
        self.x -= 1
        if self.x < 0:
            self.x = 0

    def move_down(self):
        self.x += 1
        if self.x > 3:
            self.x = 3

    def is_done(self):
        if self.x == 3 and self.y == 3:
            return True
        else:
            return False

    def get_state(self):
        return (self.x, self.y)

    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)


class Agent():
    def __init__(self):
        pass

    def select_action(self):
        coin = random.random()
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
        return action

### TD

In [2]:
def main():
    # TD
    env = GridWorld()
    agent = Agent()
    # 1. 테이블의 값을 초기화 한다.
    data = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]] 
    gamma = 1.0
    reward = -1
    alpha = 0.01

    # 5. 테이블의 값이 수렴할 때 까지 4번을 반복한다.
    for k in range(50000): 
        done = False
        # 4. 에피소드를 마칠 때 까지 2~3번을 반복한다.
        while not done:
            x, y = env.get_state()
            # 2. agent가 policy에 따라 경험을 쌓는다.
            action = agent.select_action()
            (x_prime, y_prime), reward, done = env.step(action)
            x_prime, y_prime = env.get_state()
            # 3. 상태 전이가 일어나면 테이블의 값을 업데이트 해준다.
            # 𝑇𝐷:𝑉(𝑠𝑡)←𝑉(𝑆𝑡)+𝛼(𝑟𝑡+1+𝛾𝑉(𝑠𝑡+1)−𝑉(𝑠𝑡))
            data[x][y] = data[x][y] + alpha * (reward + gamma * data[x_prime][y_prime] - data[x][y])
        env.reset()

    # 6. 수렴한 테이블의 결과를 출력해준다.
    for row in data:
        print(row)


if __name__ == '__main__':
    main()

[-56.79481916526043, -55.05562443397609, -51.89876910060371, -49.23394981558453]
[-55.102270506271076, -52.656172367695795, -47.35562614796741, -42.69929713435356]
[-51.477125961669415, -47.85166781122008, -39.7681151540909, -27.812210252522434]
[-48.48950874923934, -42.95023283065192, -30.3256593893496, 0]


# CHAPTER 6. MDP를 모를 때 optimal policy 찾기
MDP를 모를 때 optimal policy를 찾기 위해 TD를 base로 SARSA와 Q-learning을 사용한다.   

### on-policy, off-policy
behavior policy는 다음 stage로 action을 하는 policy, target policy는 다음 sample을 얻는 policy를 말한다.   
- behavior policy = target policy 인 경우 on-policy라고 한다.   
- behavior policy != target policy 인 경우 off-policy라고 한다.

off-policy를 쓰는 가장 큰 이유는 재평가가 가능하다.

### SARSA, Q-learning
- SARSA는 on-policy를 사용한다. 이름 그대로 State에서 Action을 선택하면 Reward를 받고 State'에 도착하여 거기서 다시 Action'을 선택한다.   
- Q-learning은 off-policy를 사용한다. 이는 behavior policy(행동 정책)과 target policy이 다른 경우를 말한다.   
Q_table에서 target policy로 expected 값을 얻고, target policy로 얻은 정보를 토대로 behavior policy로 action한다.

TD기반 SARSA 학습 : $q_\pi(s_t,a_t)=E_\pi(r_{r+1}+\gamma q_\pi(s_{t+1},a_{t+1})]$   
TD기반 Q-learning 학습 : $q_*(s,a)=E_{s'}[r+\gamma \underset{a'}{max}q_*(s',a')]$


### SARSA 업데이트 식
        self.q_table[x, y, a] = self.q_table[x, y, a] + 0.1 * (
                    r + self.q_table[next_x, next_y, a_prime] - self.q_table[x, y, a])

### Q러닝 업데이트 식
        self.q_table[x, y, a] = self.q_table[x, y, a] + 0.1 * (
                    r + np.amax(self.q_table[next_x, next_y, :]) - self.q_table[x, y, a])


### SARSA pseudo-code
> 1. 테이블, Q-table을 초기화 한다.
2. agent가 policy에 따라 경험을 쌓는다.
3. SARSA 정책을 통해 Q-table을 업데이트 한다.
4. 에피소드를 마칠 때 까지 2~3번을 반복한다.
5. 테이블의 값이 수렴할 때 까지 4번을 반복한다.
6. 수렴한 테이블의 결과를 출력해준다.

### grid world; env

In [3]:
import random
import numpy as np


class GridWorld():
    def __init__(self):
        self.x = 0
        self.y = 0

    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션: 아래쪽
        if a == 0:
            self.move_left()
        elif a == 1:
            self.move_up()
        elif a == 2:
            self.move_right()
        elif a == 3:
            self.move_down()

        reward = -1  # 보상은 항상 -1로 고정
        done = self.is_done()
        return (self.x, self.y), reward, done

    def move_left(self):
        if self.y == 0:
            pass
        elif self.y == 3 and self.x in [0, 1, 2]:
            pass
        elif self.y == 5 and self.x in [2, 3, 4]:
            pass
        else:
            self.y -= 1

    def move_right(self):
        if self.y == 1 and self.x in [0, 1, 2]:
            pass
        elif self.y == 3 and self.x in [2, 3, 4]:
            pass
        elif self.y == 6:
            pass
        else:
            self.y += 1

    def move_up(self):
        if self.x == 0:
            pass
        elif self.x == 3 and self.y == 2:
            pass
        else:
            self.x -= 1

    def move_down(self):
        if self.x == 4:
            pass
        elif self.x == 1 and self.y == 4:
            pass
        else:
            self.x += 1

    def is_done(self):
        if self.x == 4 and self.y == 6:  # 목표 지점인 (4,6)에 도달하면 끝난다
            return True
        else:
            return False

    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

### SARSA

In [8]:
class QAgent():
    def __init__(self):
        # 1. Q-table을 초기화 한다.
        self.q_table = np.zeros((5, 7, 4))
        self.eps = 0.9

    def select_action(self, s):
        # eps-greedy로 액션을 선택해준다
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0, 3)
        else:
            action_val = self.q_table[x, y, :]
            action = np.argmax(action_val)
        return action

    def update_table(self, transition):
        s, a, r, s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime)  # S'에서 선택할 액션 (실제로 취한 액션이 아님)
        # SARSA 업데이트 식을 이용
        self.q_table[x, y, a] = self.q_table[x, y, a] + 0.1 * (
                    r + self.q_table[next_x, next_y, a_prime] - self.q_table[x, y, a])

    def anneal_eps(self):
        self.eps -= 0.03
        self.eps = max(self.eps, 0.1)

    def show_table(self):
        q_lst = self.q_table.tolist()
        data = np.zeros((5, 7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

def main():
    env = GridWorld()
    agent = QAgent()

    # 5. 테이블의 값이 수렴할 때 까지 4번을 반복한다.
    for n_epi in range(1000):
        done = False

        s = env.reset()
        # 4. 에피소드를 마칠 때 까지 2~3번을 반복한다.
        while not done:
            # 2. agent가 policy에 따라 경험을 쌓는다.
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            # 3. SARSA 정책을 통해 Q-table을 업데이트 한다.
            agent.update_table((s, a, r, s_prime))
            s = s_prime
        agent.anneal_eps()
    
    # 6. 수렴한 테이블의 결과를 출력해준다.
    agent.show_table()


if __name__ == '__main__':
    main()

[[3. 3. 0. 3. 2. 2. 3.]
 [2. 3. 0. 2. 2. 3. 3.]
 [2. 3. 0. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 2. 0.]]


### Q-learning pseudo-code
> 1. 테이블, Q-table을 초기화 한다.
2. agent가 policy에 따라 경험을 쌓는다.
3. Q-learning 정책을 통해 Q-table을 업데이트 한다.
4. 에피소드를 마칠 때 까지 2~3번을 반복한다.
5. 테이블의 값이 수렴할 때 까지 4번을 반복한다.
6. 수렴한 테이블의 결과를 출력해준다.

### Q-learning

In [12]:
class QAgent():
    def __init__(self):
        # 1. Q-table을 초기화 한다.
        self.q_table = np.zeros((5, 7, 4))
        self.eps = 0.9

    def select_action(self, s):
        # eps-greedy로 액션을 선택해준다
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0, 3)
        else:
            action_val = self.q_table[x, y, :]
            action = np.argmax(action_val)
        return action

    def update_table(self, transition):
        s, a, r, s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime)  # S'에서 선택할 액션 (실제로 취한 액션이 아님)
        # Q러닝 업데이트 식을 이용
        self.q_table[x, y, a] = self.q_table[x, y, a] + 0.1 * (
                    r + np.amax(self.q_table[next_x, next_y, :]) - self.q_table[x, y, a])

    def anneal_eps(self):
        self.eps -= 0.01  # Q러닝에선 epsilon 이 좀더 천천히 줄어 들도록 함.
        self.eps = max(self.eps, 0.2)

    def show_table(self):
        q_lst = self.q_table.tolist()
        data = np.zeros((5, 7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)


def main():
    env = GridWorld()
    agent = QAgent()

    # 5. 테이블의 값이 수렴할 때 까지 4번을 반복한다.
    for n_epi in range(1000):
        done = False

        s = env.reset()
        # 4. 에피소드를 마칠 때 까지 2~3번을 반복한다.
        while not done:
            # 2. agent가 policy에 따라 경험을 쌓는다.
            a = agent.select_action(s) 
            s_prime, r, done = env.step(a)
            # 3. Q-learning 정책을 통해 Q-table을 업데이트 한다.
            agent.update_table((s, a, r, s_prime))
            s = s_prime
        agent.anneal_eps()

    agent.show_table()


if __name__ == '__main__':
    main()

[[3. 3. 0. 3. 3. 2. 3.]
 [2. 3. 0. 2. 2. 3. 3.]
 [3. 3. 0. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 2. 3.]
 [3. 3. 2. 1. 0. 2. 0.]]
